## Lane segmentation
This notebooks contains the code we used to train image segmentation to detect the lane markers from an RGB image

### 1. Prepare data
From carla we obtained RGB images and their respective segmentation image. We need to convert those images into a binary mask and create a dataset for them. To this end we require the following structure before hand:
- data/
    - raw/
        - rgb_images/
        - segmentation_images/


In [1]:

%load_ext autoreload
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_path not in sys.path:
    sys.path.append(root_path)


In [7]:
import re
import cv2
import numpy as np
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from tqdm import tqdm
from loguru import logger
import wandb
%autoreload 2
from DAI.cv.traffic_lane_segmentation import TrafficLaneSegmentationDataModule, TrafficLaneSegmentationModel
logger.remove()
logger.add(lambda msg: tqdm.write(msg, end=''))

2

In [3]:
data_root = os.path.abspath("../data") ## This should point to the data directory as described above.
print(data_root)
os.makedirs(os.path.join(data_root, 'images'), exist_ok=True)
os.makedirs(os.path.join(data_root, 'labels'), exist_ok=True)

/workspaces/DAI_project/data


In [23]:
traffic_lane_color = np.array(([157, 234, 50]))

image_set_length = len(os.listdir(os.path.join(data_root, 'raw', 'rgb_images')))
for image_name in tqdm(os.listdir(os.path.join(data_root, 'raw', 'rgb_images'))):
    image_number = re.findall('image(\d+).png', image_name)[0]
    rgb_image_path = os.path.join(data_root, 'raw', 'rgb_images', image_name)
    seg_image_path = os.path.join(data_root, 'raw', 'segmentation_images',f'seg{image_number}.png')
    rgb_image = cv2.imread(rgb_image_path)
    seg_image = cv2.imread(seg_image_path)
    if rgb_image is None or seg_image is None:
        logger.warning(f"skipping {rgb_image_path}")
        continue
    mask: np.ndarray = np.any(seg_image == traffic_lane_color, axis=-1)
    binary_segmentation_image = (mask*255).astype(np.uint8)
    cv2.imwrite(os.path.join(data_root, 'images', image_name), rgb_image)
    cv2.imwrite(os.path.join(data_root, 'labels', f"segm{image_number}.png"), binary_segmentation_image)


 17%|█▋        | 151/881 [00:11<00:50, 14.43it/s]

2024-12-02 15:16:14.449 | WARNING  | __main__:<module>:11 - skipping /workspaces/DAI_project/data/raw/rgb_images/image325.png


 21%|██▏       | 188/881 [00:14<00:44, 15.44it/s]

2024-12-02 15:16:17.133 | WARNING  | __main__:<module>:11 - skipping /workspaces/DAI_project/data/raw/rgb_images/image904.png


 23%|██▎       | 203/881 [00:15<00:44, 15.17it/s]

...

 79%|███████▊  | 692/881 [00:51<00:12, 14.65it/s]

2024-12-02 15:16:54.052 | WARNING  | __main__:<module>:11 - skipping /workspaces/DAI_project/data/raw/rgb_images/image94.png
2024-12-02 15:16:54.216 | WARNING  | __main__:<module>:11 - skipping /workspaces/DAI_project/data/raw/rgb_images/image464.png


 83%|████████▎ | 730/881 [00:53<00:10, 15.05it/s]

2024-12-02 15:16:56.749 | WARNING  | __main__:<module>:11 - skipping /workspaces/DAI_project/data/raw/rgb_images/image55.png


100%|██████████| 881/881 [01:04<00:00, 13.58it/s]


### 2 Prepare Model
We use the torchvision DeepLabV3 segmentation model which we wrapped with a lightning module

In [5]:

data_loader = TrafficLaneSegmentationDataModule(data_root)
model = TrafficLaneSegmentationModel()

/workspaces/DAI_project/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/workspaces/DAI_project/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### 3 Train model
We setup training with Weights and biases early stopping and model checkpointing

In [8]:
save_callback = ModelCheckpoint('./models', 'best.pt', monitor='val_loss')
early_stopper = EarlyStopping(monitor='val_loss', patience=5)
logging = WandbLogger(project='Traffic-lane-segmentation', name='Test_run')
trainer = L.Trainer(accelerator='auto', callbacks=[save_callback, early_stopper])
trainer.fit(model, datamodule=data_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/workspaces/DAI_project/.venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/workspaces/DAI_project/.venv/lib/python3.8/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
Missing logger folder: /workspaces/DAI_project/notebooks/lightning_logs

  | Name  | Type              | Params | Mode 
---------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid(s) 20950) exited unexpectedly